In [15]:
import tensorflow.compat.v1 as tf

In [16]:
print(tf.__version__)

2.0.0


In [19]:
tf.disable_eager_execution()

In [58]:
h = tf.constant("Hello")
w = tf.constant(" World!")

hw = h + w
#tf.print(hw)

with tf.Session() as sess:
      ans = sess.run(hw)

ans



b'Hello World!'

In [20]:
#creating computation graph
x = tf.Variable(3,name='x')
y = tf.Variable(3,name='y')

f = x ** 2 * y + y + 2
f

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


<tf.Tensor 'add_9:0' shape=() dtype=int32>

### TensorFlow session

In [27]:
#tensorflow session
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)

result = sess.run(f)
print(result)
sess.close()

#with
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

print(result)

#global_variable_initializer
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
print(result)



    

32
32
32


## Linear Regression with TensorFlow

In [54]:
from sklearn.datasets import fetch_california_housing
import numpy as np


housing = fetch_california_housing()
n,m = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((n,1)),housing.data]


#normal equation with tensor flow
X = tf.constant(housing_data_plus_bias,name ='X')
y = tf.constant(housing.target.reshape(-1,1),name ='y')

XT = tf.transpose(X)


theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)
with tf.Session() as sess:
    result = theta.eval()
print('tensor',result)

#pure numpy
X = housing_data_plus_bias
y = housing.target.reshape(-1,1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
print('theta_numpy',theta_numpy)
#or theta_numpy = np.matmul(np.matmul(np.linalg.inv(np.matmul(X.T,X)),X.T),y)



tensor [[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]
theta_numpy [[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


### gradient descent


In [74]:
# gradient descent
from sklearn.preprocessing import StandardScaler

# gd requires scaling
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_bias = np.c_[np.ones((n,1)),scaled_housing_data]

# manualy
tf.reset_default_graph()

n_epoch = 1000
learning_rate = 0.01

print(m)
X = tf.constant(scaled_housing_data_bias,dtype=tf.float32,name='X')
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name='y')
theta = tf.Variable(tf.random_uniform([m + 1,1],-1,1,seed=42),name='theta')
y_pred = tf.matmul(X,theta,name='predictions')
    
# init_op = tf.global_variables_initializer()
# with tf.Session() as sess:
#     sess.run(init_op) 
#     print ('theta',sess.run(theta))
#     print('y_pred',sess.run(y_pred))
    
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse')
gradients = 2 /n * tf.matmul(tf.transpose(X),error)

traning_op = tf.assign(theta,theta-learning_rate*gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epoch):
        if epoch % 100 == 0:
            print ('epoch',epoch,'MSE = ',mse.eval())
        sess.run(traning_op)
        
    best_theta = theta.eval()
print('best_theta')
print(best_theta)
            

8
epoch 0 MSE =  2.7544265
epoch 100 MSE =  0.632222
epoch 200 MSE =  0.5727804
epoch 300 MSE =  0.5585008
epoch 400 MSE =  0.54907006
epoch 500 MSE =  0.54228795
epoch 600 MSE =  0.5373791
epoch 700 MSE =  0.533822
epoch 800 MSE =  0.53124255
epoch 900 MSE =  0.5293705
best_theta
[[ 2.0685523e+00]
 [ 7.7407807e-01]
 [ 1.3119239e-01]
 [-1.1784512e-01]
 [ 1.6477819e-01]
 [ 7.4407941e-04]
 [-3.9194509e-02]
 [-8.6135662e-01]
 [-8.2347971e-01]]


### Gradient Descent Optimizer

In [76]:
# gradient descent with optimizer
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_bias = np.c_[np.ones((n,1)),scaled_housing_data]

tf.reset_default_graph()
n_epoch = 1000
learning_rate = 0.01


X = tf.constant(scaled_housing_data_bias,dtype=tf.float32,name='X')
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name='y')
theta = tf.Variable(tf.random_uniform([m + 1,1],-1,1,seed=42),name='theta')
y_pred = tf.matmul(X,theta,name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse')

gradients_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

traning_op = gradients_optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epoch):
        if epoch % 100 == 0:
            print ('epoch',epoch,'MSE = ',mse.eval())
        sess.run(traning_op)
        
    best_theta = theta.eval()
print('best_theta')
print(best_theta)
            

epoch 0 MSE =  2.7544265
epoch 100 MSE =  0.6322221
epoch 200 MSE =  0.5727804
epoch 300 MSE =  0.5585008
epoch 400 MSE =  0.54907006
epoch 500 MSE =  0.542288
epoch 600 MSE =  0.5373791
epoch 700 MSE =  0.533822
epoch 800 MSE =  0.53124255
epoch 900 MSE =  0.5293705
best_theta
[[ 2.06855249e+00]
 [ 7.74078071e-01]
 [ 1.31192371e-01]
 [-1.17845125e-01]
 [ 1.64778188e-01]
 [ 7.44073535e-04]
 [-3.91945057e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


### Momentum Optimizer

In [77]:
# gradient descent with optimizer
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_bias = np.c_[np.ones((n,1)),scaled_housing_data]

tf.reset_default_graph()
n_epoch = 1000
learning_rate = 0.01


X = tf.constant(scaled_housing_data_bias,dtype=tf.float32,name='X')
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name='y')
theta = tf.Variable(tf.random_uniform([m + 1,1],-1,1,seed=42),name='theta')
y_pred = tf.matmul(X,theta,name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse')

gradients_optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)

traning_op = gradients_optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epoch):
        if epoch % 100 == 0:
            print ('epoch',epoch,'MSE = ',mse.eval())
        sess.run(traning_op)
        
    best_theta = theta.eval()
print('best_theta')
print(best_theta)
            

epoch 0 MSE =  2.7544265
epoch 100 MSE =  0.52731603
epoch 200 MSE =  0.5244147
epoch 300 MSE =  0.5243281
epoch 400 MSE =  0.5243219
epoch 500 MSE =  0.52432114
epoch 600 MSE =  0.524321
epoch 700 MSE =  0.524321
epoch 800 MSE =  0.524321
epoch 900 MSE =  0.524321
best_theta
[[ 2.068558  ]
 [ 0.8296165 ]
 [ 0.11875115]
 [-0.26552156]
 [ 0.30569187]
 [-0.00450317]
 [-0.03932614]
 [-0.8998925 ]
 [-0.87054735]]



### mini batch gradient descent

In [79]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_bias = np.c_[np.ones((n,1)),scaled_housing_data]



tf.reset_default_graph()
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(n / batch_size))

learning_rate = 0.01


X = tf.placeholder(tf.float32,shape=(None, m + 1),name='X')
y = tf.placeholder(tf.float32,shape=(None, 1),name='y')

theta = tf.Variable(tf.random_uniform([m + 1,1],-1,1,seed=42),name='theta')
y_pred = tf.matmul(X,theta,name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse')

gradients_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

traning_op = gradients_optimizer.minimize(mse)
init = tf.global_variables_initializer()



def fetch_batch(epoch,batch_index,batch_size):
    indices = np.random.randint(n,size=batch_size)
    x_batch = scaled_housing_data_bias[indices]
    y_batch = housing.target.reshape(-1,1)[indices]
    return x_batch,y_batch


np.random.seed(0)

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        print ('epoch',epoch,'MSE = ',
               mse.eval(feed_dict={X: scaled_housing_data_bias,
                                   y: housing.target.reshape(-1,1)}))
        for batch_index in range(n_batches):
            X_batch,y_batch = fetch_batch(epoch,batch_index,batch_size)
            sess.run(traning_op,feed_dict={X: X_batch,y: y_batch})
        
        
    best_theta = theta.eval()
    
print('best_theta')
print(best_theta)
            

epoch 0 MSE =  2.7544265
epoch 1 MSE =  0.87899464
epoch 2 MSE =  0.5497306
epoch 3 MSE =  0.5361563
epoch 4 MSE =  0.53294533
epoch 5 MSE =  0.5291698
epoch 6 MSE =  0.52780205
epoch 7 MSE =  0.5273184
epoch 8 MSE =  0.5337286
epoch 9 MSE =  0.5291435
best_theta
[[ 2.0685194 ]
 [ 0.8433898 ]
 [ 0.11697166]
 [-0.28834364]
 [ 0.253429  ]
 [-0.00420929]
 [ 0.02694976]
 [-0.8980864 ]
 [-0.8800862 ]]
